In [1]:
from collections.abc import Iterable
import os

from dandi.dandiapi import DandiAPIClient
from datetime import datetime, timedelta
from dateutil import tz
import h5py
from hdmf.backends.hdf5.h5_utils import H5DataIO
from hdmf.container import Container
from hdmf.data_utils import DataChunkIterator
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pynwb import load_namespaces, get_class, register_class, NWBFile, TimeSeries, NWBHDF5IO
from pynwb.file import MultiContainerInterface, NWBContainer, Device, Subject
from pynwb.ophys import ImageSeries, OnePhotonSeries, OpticalChannel, ImageSegmentation, PlaneSegmentation, Fluorescence, DfOverF, CorrectedImageStack, MotionCorrection, RoiResponseSeries, ImagingPlane
from pynwb.core import NWBDataInterface
from pynwb.epoch import TimeIntervals
from pynwb.behavior import SpatialSeries, Position
from pynwb.image import ImageSeries
import remfile
import scipy.io as sio
import skimage.io as skio
from tifffile import TiffFile

# ndx_mulitchannel_volume is the novel NWB extension for multichannel optophysiology in C. elegans
from ndx_multichannel_volume import CElegansSubject, OpticalChannelReferences, OpticalChannelPlus, ImagingVolume, VolumeSegmentation, MultiChannelVolume, MultiChannelVolumeSeries

In [11]:
import os
import PyQt6.QtCore
os.environ["QT_API"] = "pyqt6"

In [3]:
dandi_id = '000541' #EY dataset

AVA_traces = []
SMDV_traces = []
AWC_traces = []
ASH_traces = []
RID_traces = []

with DandiAPIClient() as client:
    dandiset = client.get_dandiset(dandi_id, 'draft')
    for asset in dandiset.get_assets():
        s3_url = asset.get_content_url(follow_redirects=1, strip_query=True)
        file = remfile.File(s3_url)

        with h5py.File(file, 'r') as f:
            with NWBHDF5IO(file=f, mode='r', load_namespaces=True) as io:
                read_nwb = io.read()
                identifier = read_nwb.identifier
                print(identifier)
                seg = read_nwb.processing['NeuroPAL']['NeuroPALSegmentation']['NeuroPALNeurons'].voxel_mask[:]
                labels = read_nwb.processing['NeuroPAL']['NeuroPALSegmentation']['NeuroPALNeurons']['ID_labels'][:]
                rate = read_nwb.acquisition['CalciumImageSeries'].rate
                fluor = read_nwb.processing['CalciumActivity']['SignalRawFluor']['SignalCalciumImResponseSeries'].data[:]
                calc_labels = read_nwb.processing['CalciumActivity']['NeuronIDs'].labels[:]

        index_AVA = np.argwhere((np.asarray(calc_labels)=='AVAR') | (np.asarray(calc_labels)=='AVAL'))
        index_SMDV = np.argwhere((np.asarray(calc_labels)=='SMDVR') | (np.asarray(calc_labels)=='SMDVL'))
        index_AWC = np.argwhere((np.asarray(calc_labels)=='AWCR') | (np.asarray(calc_labels)=='AWCL'))
        index_ASH = np.argwhere((np.asarray(calc_labels)=='ASHR') | (np.asarray(calc_labels)=='ASHL'))
        index_RID = np.argwhere(np.asarray(calc_labels)=='RID')
        AVA_traces = AVA_traces + [np.squeeze(fluor[:,index]/np.mean(fluor[:,index])) for index in index_AVA if index is not None]
        SMDV_traces = SMDV_traces + [np.squeeze(fluor[:,index]/np.mean(fluor[:,index])) for index in index_SMDV if index is not None]
        AWC_traces = AWC_traces + [np.squeeze(fluor[:, index]/np.mean(fluor[:,index])) for index in index_AWC if index is not None]
        ASH_traces = ASH_traces + [np.squeeze(fluor[:,index]/np.mean(fluor[:,index])) for index in index_ASH if index is not None]
        RID_traces = RID_traces + [np.squeeze(fluor[:,index]/np.mean(fluor[:,index])) for index in index_RID if index is not None]
    

20190924_02
20190924_01
20190925_04
20190924_03
20190925_01
20190928_03
20190928_05
20190928_01
20190928_08
20190928_11
20190929_02
20190928_13
20190929_05
20190929_03
20190928_07
20190929_07
20191104_10
20191030_03
20191104_08
20190929_06
20191030_07


'\ntraces = np.transpose(fluor)\nlabels = ["".join(label) for label in labels]\n\nplt.rcParams.update(plt.rcParamsDefault)\n\ndef plot_traces(traces, rate, labels, selected):\n\n    plt.rcParams.update({\'font.size\':20})\n\n    seconds = traces.shape[1]//rate\n\n    fig, axs = plt.subplots(len(selected),1, figsize=(5,6))\n\n    for i, neuron in enumerate(selected):\n        index = np.argwhere(np.asarray(labels)==neuron)\n        trace = traces[np.squeeze(index),:]\n\n        axs[i].plot(np.linspace(0,seconds,traces.shape[1]), trace)\n        axs[i].set_ylabel(r\'$\\Delta$F/F\')\n        axs[i].set_xlim(0,seconds)\n        axs[i].set_yticks([])\n        axs[i].set_title(neuron, loc=\'left\')\n\n        axs[i].spines[\'right\'].set_visible(False)\n        axs[i].spines[\'top\'].set_visible(False)\n\n    axs[4].set_xlabel(\'time (seconds)\')\n\n    plt.tight_layout()\n    plt.show()\n\nplot_traces(traces, rate, labels, [\'AVAR\', \'SMDVR\', \'AWCR\',\'RID\', \'ASHR\'])\n'

In [18]:
%matplotlib qt

def plot_traces_fourier(traces, labels, rate):

    fig, axs = plt.subplots(len(traces),2)

    for i, neuron_trace in enumerate(traces):
        for j, trace in enumerate(neuron_trace):
            trace_cropped = trace[:936] #truncate so all traces have the same length
            time_steps = np.linspace(0,936/4.0,936)
            axs[i,0].plot(time_steps,trace_cropped, alpha=0.5)

            n = len(trace_cropped)
            yf = np.fft.rfft(trace_cropped)
            xf = np.fft.rfftfreq(n, d=1/rate)

            axs[i,1].plot(xf, yf, alpha=0.5)
        
        axs[i,0].set_title('Time domain - '+labels[i])
        axs[i,1].set_title('Frequency domain - '+labels[i])

traces = [AVA_traces, SMDV_traces, AWC_traces, ASH_traces, RID_traces]
labels = ['AVA', 'SMDV', 'AWC', 'ASH', 'RID']
plot_traces_fourier(traces, labels, rate)

/Applications/anaconda3/envs/NWB-dev/lib/python3.11/site-packages/matplotlib/cbook/__init__.py:1340: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)
